1.1 – Đọc toàn bộ dữ liệu thô từ các file .csv

In [1]:
import pandas as pd
import numpy as np

duong_dan_goc = '../data_raw/' 
df_thong_tin = pd.read_csv(duong_dan_goc + 'sensor_info.csv')
df_doc_cam_bien = pd.read_csv(duong_dan_goc + 'sensor_readings.csv')
df_bao_tri = pd.read_csv(duong_dan_goc + 'maintenance_log.csv')

1.2 – Làm sạch sensor_id, machine_id, sensor_type

In [2]:
df_thong_tin['sensor_id'] = (df_thong_tin['sensor_id'].astype(str).str.strip().str.upper())

df_thong_tin['machine_id'] = (df_thong_tin['machine_id'].astype(str).str.strip().str.upper())

stype_raw = df_thong_tin['sensor_type'].astype(str).str.strip()
stype_low = stype_raw.str.lower()
df_thong_tin.loc[stype_low.str.contains('temp'), 'sensor_type'] = 'Temperature'
df_thong_tin.loc[stype_low.str.contains('vib'), 'sensor_type'] = 'Vibration'
df_thong_tin.loc[stype_low.str.contains('curr'), 'sensor_type'] = 'Current'

1.3 – Chuẩn hóa location

In [3]:
loc_raw = df_thong_tin['location'].astype(str).str.strip()
loc_key = loc_raw.str.lower().str.replace(' ', '')

In [4]:
df_thong_tin.loc[loc_key.str.contains('line1'), 'location'] = 'Line 1'
df_thong_tin.loc[loc_key.str.contains('khua'), 'location'] = 'Khu A'

In [5]:
mask_other = (loc_key.str.contains('line1') | loc_key.str.contains('khua'))
df_thong_tin.loc[mask_other, 'location'] = loc_raw[mask_other].str.title()

In [6]:
df_thong_tin.head()

,sensor_id,machine_id,location,sensor_type
0,SEN01,MC01,Line1,Current
1,SEN02,MC04,Line1,Temperature
2,SEN03,MC04,Khu A,Vibration
3,SEN04,MC05,Line1,Vibration
4,SEN05,MC05,Khu A,Vibration


1.4 – Chuẩn hóa timestamp cùng múi giờ & định dạng thời gian

In [7]:
ts_str = df_doc_cam_bien['timestamp'].astype(str).str.strip()

mask_gach = ts_str.str.contains('-')   
mask_slash = ts_str.str.contains('/')  

df_doc_cam_bien['timestamp'] = pd.NaT

In [8]:
df_doc_cam_bien.loc[mask_gach, 'timestamp'] = pd.to_datetime(ts_str[mask_gach],format='%Y-%m-%d %H:%M')
df_doc_cam_bien.loc[mask_slash, 'timestamp'] = pd.to_datetime(ts_str[mask_slash],format='%d/%m/%Y %H:%M')

1.5 – Làm sạch value và unit (lọc text, ký tự lạ, xử lý giá trị bất thường)

In [9]:
df_doc_cam_bien['value'] = df_doc_cam_bien['value'].astype(str).str.strip()

df_doc_cam_bien['value'] = df_doc_cam_bien['value'].replace(['error', 'ERROR', 'na', 'NA', 'NaN', 'N/A', '-', 'None'],np.nan)

df_doc_cam_bien['value'] = df_doc_cam_bien['value'].str.replace('C', '')

df_doc_cam_bien['value'] = df_doc_cam_bien['value'].astype(float)

mask_neg_temp = (df_doc_cam_bien['value'] < 0) & (df_doc_cam_bien['unit'] == '°C')
df_doc_cam_bien.loc[mask_neg_temp, 'value'] = np.nan

In [10]:
df_doc_cam_bien['unit'] = df_doc_cam_bien['unit'].astype(str).str.strip()
u_raw = df_doc_cam_bien['unit']
u_low = u_raw.str.lower()

df_doc_cam_bien.loc[u_low.str.contains('mm/s'), 'unit'] = 'mm/s'

mask_amp = (u_low.str.contains('amp')) | (u_low == 'a')
df_doc_cam_bien.loc[mask_amp, 'unit'] = 'A'

df_doc_cam_bien.loc[u_low.str.contains('c'), 'unit'] = '°C'


In [11]:
df_doc_cam_bien.head()

,sensor_id,timestamp,value,unit
0,SEN08,2024-01-08 12:23:00,NaN,°C
1,sen03,2024-01-08 18:46:00,80.0,A
2,SEN09,2024-01-01 20:33:00,25.5,mm/s
3,SEN08,2024-01-07 17:47:00,45.0,°C
4,SEN09,2024-01-09 12:37:00,25.5,A


1.6 – Chuẩn hóa maintenance_date, issue_type và downtime_hours

In [12]:
df_bao_tri['machine_id'] = (df_bao_tri['machine_id'].astype(str).str.strip().str.upper())

In [13]:
ngay_str = df_bao_tri['maintenance_date'].astype(str).str.strip()

mask_gach = ngay_str.str.contains('-')    
mask_slash = ngay_str.str.contains('/')   

df_bao_tri['maintenance_date'] = pd.NaT

df_bao_tri.loc[mask_gach, 'maintenance_date'] = pd.to_datetime(ngay_str[mask_gach],format='%Y-%m-%d')
df_bao_tri.loc[mask_slash, 'maintenance_date'] = pd.to_datetime(ngay_str[mask_slash],format='%d/%m/%Y')

In [14]:
df_bao_tri['issue_type'] = df_bao_tri['issue_type'].astype(str).str.strip()
issue_low = df_bao_tri['issue_type'].str.lower()

df_bao_tri.loc[issue_low.str.contains('over'), 'issue_type'] = 'Overheat'
df_bao_tri.loc[issue_low.str.contains('vib'), 'issue_type'] = 'Vibration'
df_bao_tri.loc[issue_low.str.contains('elect'), 'issue_type'] = 'Electrical'

In [15]:
df_bao_tri['downtime_hours'] = df_bao_tri['downtime_hours'].astype(str).str.strip()

df_bao_tri['downtime_hours'] = df_bao_tri['downtime_hours'].replace(['na', 'NA', 'NaN', 'N/A', '-', 'None', ''],np.nan)
df_bao_tri['downtime_hours'] = df_bao_tri['downtime_hours'].str.replace('h', '')
df_bao_tri['downtime_hours'] = df_bao_tri['downtime_hours'].astype(float)

In [16]:
df_bao_tri.head()

,machine_id,maintenance_date,issue_type,downtime_hours
0,MC04,2024-02-05,Overheat,1.5
1,MC01,2024-01-10,Electrical,NaN
2,MC04,2024-01-08,Vibration,3.0
3,MC02,2024-02-01,Electrical,2.0
4,MC01,2024-01-08,Vibration,1.5
